# Kriging

In [ ]:
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging

unique_dates = krig_df['datetime'].unique()
meteo_list = ['wind_speed', 'air_temp', 'rel_humidity', 'dew_point']

num_dates = len(unique_dates)
c = 0

for date in unique_dates:
    print(date)
    for variable in meteo_list:

        meteo_stations = krig_df[krig_df['datetime'] == date]
        meteo_stations = meteo_stations[meteo_stations[variable].notnull()]
        
        grid_lon = np.linspace(0.0, 360.0, 7)
        grid_lat = np.linspace(-90.0, 90.0, 7)
        
        lon = meteo_stations['lon']
        lat = meteo_stations['lat']
        z = meteo_stations[variable]
        
        OK = OrdinaryKriging(lon, lat, z, variogram_model='linear', verbose=False,
                     enable_plotting=False, coordinates_type='geographic')
        
        date_df = main_df[main_df['datetime'] == date]
        date_df = date_df[date_df[variable].isnull()]
        
        for idx, row in date_df.iterrows():
            lat_long = np.asarray(row[['lat', 'long']])

            z2, ss2 = OK.execute('grid', lon, lat)
            print(ss2)
            
            
            plt.scatter(meteo_stations['lat'], meteo_stations['lon'])
            plt.show()
            main_df.at[idx, variable] = y_pred
        break   
    break
    c += 1
    if c % 1000 == 0:
        print('Change for {} of the {} unique dates'.format(c, num_dates))

In [ ]:
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging

unique_dates = krig_df['datetime'].unique()
meteo_list = ['wind_speed', 'air_temp', 'rel_humidity', 'dew_point']

num_dates = len(unique_dates)
c = 0

for date in unique_dates:

    for variable in meteo_list:

        meteo_stations = krig_df[krig_df['datetime'] == date]
        meteo_stations = meteo_stations[meteo_stations[variable].notnull()]
        print(meteo_stations)
        x = meteo_stations[['lat', 'lon']].as_matrix()
        y = meteo_stations[variable]
        
        GPR = GaussianProcessRegressor()
        GPR.fit(x, y)
        
        date_df = main_df[main_df['datetime'] == date]
        date_df = date_df[date_df[variable].isnull()]
        
        for idx, row in date_df.iterrows():
            lat_long = np.asarray(row[['lat', 'long']])

            y_pred = GPR.predict(lat_long.reshape(1, -1))
            
            if y_pred < 0 and variable == 'wind_speed':
                xy = [[x1, y1] for x1, y1 in zip(x,y)]
#                 print(xy)
                print(lat_long)
                print(y_pred, idx, variable)
            
                plt.scatter(meteo_stations['lat'], meteo_stations['lon'])
                plt.show()
                main_df.at[idx, variable] = y_pred
  


    c += 1
    if c % 1000 == 0:
        print('Change for {} of the {} unique dates'.format(c, num_dates))